In [2]:
from flask import Flask, request, jsonify
from twelvedata import TDClient
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import numpy as np

app = Flask(__name__)

# Initialize Twelve Data client
td = TDClient(apikey="your_api_key_here")

def build_lstm_model(data):
    # Convert data to a numpy array and reshape for the scaler
    data = np.array(data).reshape(-1, 1)
    # Normalize data
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_normalized = scaler.fit_transform(data)
    X = data_normalized[:-1]
    y = data_normalized[1:]
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    
    # Define LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, y, epochs=100, batch_size=32, validation_split=0.1)
    
    # Return both the model and the scaler for later use
    return model, scaler

@app.route('/predict/short-term')
def predict_short_term():
    symbol = 'AAPL'
    interval = '30min'
    outputsize = 500
    
    ts = td.time_series(symbol=symbol, interval=interval, outputsize=outputsize)
    historical_data = ts.as_json()
    
    closing_prices = [float(data_point['close']) for data_point in historical_data]
    lstm_model, scaler = build_lstm_model(closing_prices)
    
    # Preparing the last data point for prediction
    last_datapoint = scaler.transform(np.array([closing_prices[-1]]).reshape(-1, 1))
    predicted_data_normalized = lstm_model.predict(last_datapoint.reshape(1, 1, 1))
    predicted_data = scaler.inverse_transform(predicted_data_normalized)
    
    return jsonify(predicted_data.tolist())

@app.route('/predict/medium-term')
def predict_medium_term():
    symbol = 'AAPL'
    interval = '1h'
    outputsize = 1000
    
    ts = td.time_series(symbol=symbol, interval=interval, outputsize=outputsize)
    historical_data = ts.as_json()
    
    closing_prices = [float(data_point['close']) for data_point in historical_data]
    lstm_model, scaler = build_lstm_model(closing_prices)
    
    # Preparing the last data point for prediction
    last_datapoint = scaler.transform(np.array([closing_prices[-1]]).reshape(-1, 1))
    predicted_data_normalized = lstm_model.predict(last_datapoint.reshape(1, 1, 1))
    predicted_data = scaler.inverse_transform(predicted_data_normalized)
    
    return jsonify(predicted_data.tolist())

if __name__ == "__main__":
    app.run(debug=True)


ImportError: dlopen(/Users/tharidhu/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/platform/_pywrap_cpu_feature_guard.so, 0x0002): Library not loaded: @rpath/libtensorflow_cc.2.dylib
  Referenced from: <259C265C-4CD3-3142-818C-957EA0E17164> /Users/tharidhu/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so
  Reason: tried: '/Users/tharidhu/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/../../_solib_darwin_x86_64/libtensorflow_cc.2.dylib' (no such file), '/Users/tharidhu/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utensorflow_Uinternal.so_Ucclib___Utensorflow/libtensorflow_cc.2.dylib' (no such file), '/Users/tharidhu/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/libtensorflow_cc.2.dylib' (no such file), '/Users/tharidhu/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/../libtensorflow_cc.2.dylib' (no such file), '/Users/